In [1]:
! pip3 install pandas
! pip3 install numpy
! pip3 install plotly
! pip3 install emoji-country-flag
! pip3 install dash
! pip3 install jupyter-dash


In [2]:
# Turn off reloader if inside Jupyter

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import flag
from ast import literal_eval

In [6]:
df = pd.read_csv('spotify_phonk.csv')
df.head()

,Unnamed: 0,track_id,name,country,date,position,streams,artists,artist_genres,duration,explicit,counter
0,295780,4wHkfOTO5XpTrVn1bHxR18,Dream Space,ua,2021-10-07,190,24172,['DVRST'],['drift phonk'],171466,True,1
1,325679,5takBO1frKuNNEFXPy35wX,Tony (feat. DJ Metys & Brenk Sinatra),sk,2017-11-16,88,5633,"['Pil C', 'DJ Metys', 'Brenk Sinatra']","['jazz boom bap', 'czsk hip hop', 'phonk', 'ch...",220101,True,1
2,325680,5takBO1frKuNNEFXPy35wX,Tony (feat. DJ Metys & Brenk Sinatra),sk,2017-11-23,38,11162,"['Pil C', 'DJ Metys', 'Brenk Sinatra']","['jazz boom bap', 'czsk hip hop', 'phonk', 'ch...",220101,True,1
3,325681,5takBO1frKuNNEFXPy35wX,Tony (feat. DJ Metys & Brenk Sinatra),sk,2017-11-30,93,5929,"['Pil C', 'DJ Metys', 'Brenk Sinatra']","['jazz boom bap', 'czsk hip hop', 'phonk', 'ch...",220101,True,1
4,325682,5takBO1frKuNNEFXPy35wX,Tony (feat. DJ Metys & Brenk Sinatra),sk,2017-12-07,169,4089,"['Pil C', 'DJ Metys', 'Brenk Sinatra']","['jazz boom bap', 'czsk hip hop', 'phonk', 'ch...",220101,True,1


In [7]:
df_date = df.groupby('date').agg(
    {'counter':sum,
     'streams':sum}
).reset_index()

In [7]:
df_artist = df
df_artist['artists'] = df_artist['artists'].apply(literal_eval)
df_artist = df.explode('artists').groupby('artists').agg(
    {
        'counter' : sum,
        'streams' : sum
    }
).reset_index()
df_artist['artists'] = df_artist['artists'].astype(str)
df_artist = df_artist.nlargest(10, 'streams')
df_artist

,artists,counter,streams
27,Kordhell,605,136876606
62,Yung Hurn,349,69153671
58,Ufo361,127,30743498
10,DVRST,551,27937062
35,Nimo,106,24634807
11,Dxrk ダーク,371,23453973
65,niteboi,210,23196801
26,Kingpin Skinny Pimp,232,16054813
45,PlayaPhonk,342,16010376
50,SW43 L33 D1DO,25,13279699


In [8]:
df_country = df.groupby('country').agg(
    {
        'counter' : sum,
        'streams' : sum
    }
).reset_index()
df_country = df_country.nlargest(10, 'streams')
df_country['country'] = df_country.country.apply(
    lambda x: f'{flag.flag(x)} {x.upper()}'
)

In [9]:
df_explicit = df.groupby('explicit').agg(
    {
        'counter' : sum
    }
)

In [10]:


# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=df_date['date'], y=df_date['counter'], name="Number of tracks", line_color="black", fill='tozeroy'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_date['date'], y=df_date['streams'], name="Number of Steams", line_color="#00D56C", fill='tozeroy'),
    secondary_y=True,
)

fig.update_layout(
    title="Phonk popularity on Spotify",
    legend=dict(yanchor="bottom", y=0.7, xanchor="left", x=0.05)
)




In [11]:
trace_artist = go.Scatter(
    x=df_artist["counter"],
    y=df_artist["streams"], 
    mode = 'markers',
    
    marker_size= df_artist["counter"]/5,
    marker_color= df_artist["streams"],
    
)
fig2 = go.Figure()
fig2.add_trace(trace_artist)
fig2.update_layout(
    showlegend=False,
    title = "Top listened Phonk tracks over artists on spotify"
)

In [12]:
import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()
app.layout = html.Div(
    children=[
        html.Div(
            dcc.Graph(
                figure=fig
            )
        ),
        html.Div(
            dcc.Graph(
                figure=fig2
            )
        ),
    ]
)


app.run_server(debug=True, use_reloader=False)  

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


/var/folders/rg/llsw209n1tvdytqt25z1bk2m0000gn/T/ipykernel_97297/4024946788.py:2: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

/var/folders/rg/llsw209n1tvdytqt25z1bk2m0000gn/T/ipykernel_97297/4024946788.py:3: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [ ]:

fig2.show()

In [ ]:
fig = px.scatter(df_country, x="counter", y="streams", color="country",
                 size='streams', text="country",
                 labels={
                     "counter": "Number of tracks",
                     "streams": "Number of streams",
                 })
fig.update_layout(
    showlegend=False,
    title = "Top listened Phonk tracks over country on spotify"
)
fig.show()

In [ ]:

fig = go.Figure(data=[go.Pie(labels=['explicit', 'not explicit'], values=df_explicit['counter'], pull=[0, 0.2])])
fig.show()